In [1]:
import torch
from torch import nn

In [2]:
class Head(nn.Module):
    def __init__(self, n_embds, head_size):
        super().__init__()
        self.key = nn.Linear(n_embds, head_size, bias=False)
        self.query = nn.Linear(n_embds, head_size, bias=False)
        self.value = nn.Linear(n_embds, head_size, bias=False)
        self.head_size = head_size
        
        self.register_buffer('tril', torch.tril(torch.ones((T,T))))
        
    def forward(self, x):
        # Self attention since query, keys and values comes from the same input x
        B,T,C = x.shape
        k = self.key(x) # B,T,head_size
        q = self.query(x) # B,T,head_size
        v = self.value(x) # B,T,head_size
        
        wei = q @ k.transpose(-2, -1) * self.head_size**-0.5 # (B,T,head_size) @ (B,head_size,T) = (B,T,T)
        wei = wei.masked_fill(self.tril[:T, :T]==0, float('-inf'))
        wei = torch.softmax(wei, dim=-1)
        
        return wei @ v
        

In [3]:
class MultiHead(nn.Module):
    def __init__(self, num_heads, n_embds, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(n_embds, head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(num_heads * head_size, n_embds)
            
    def forward(self, x):
        x = torch.cat([h(x) for h in self.heads], dim=-1)
        x = self.proj(x)
        return x

In [4]:
class FeedFoward(nn.Module):
    """Just a MLP"""
    def __init__(self, n_embds):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embds, n_embds),
            nn.ReLU(),
            nn.Linear(n_embds, n_embds) # works as a projection layer
        )
    def forward(self, x):
        return self.net(x)

In [5]:
class Block(nn.Module):
    def __init__(self, n_embds, n_heads):
        super().__init__()
        head_size = n_embds // n_heads
        self.attn = MultiHead(n_heads, n_embds, head_size)
        self.ff = FeedFoward(n_embds)
        self.ln1 = nn.LayerNorm(n_embds)
        self.ln2 = nn.LayerNorm(n_embds)
        
    def forward(self, x):
        # Note the residual connection
        x = x + self.attn(self.ln1(x))
        x = x + self.ff(self.ln2(x))
        return x

In [7]:
class GPT(nn.Module):
    def __init__(self, n_embds, n_heads, head_size, vocab_size, block_size):
        super().__init__()
        self.token_emb = nn.Embedding(vocab_size, n_embds)
        self.pos_emb = nn.Embedding(block_size, n_embds)
        self.blocks = nn.Sequential(
            Block(n_embds, n_heads),
            Block(n_embds, n_heads),
            Block(n_embds, n_heads),
            nn.LayerNorm(n_embds),
        )
        self.proj = nn.Linear(n_embds, vocab_size)
    
    def forward(self, idx):
        B, T = idx.shape
        token_emb = self.token_emb(idx) # B,T,n_embds
        pos_emb = self.pos_emb(torch.arange(T, device=idx.device)) # T,n_embds
        
        x = token_emb + pos_emb
        x = self.blocks(x)
        logits = self.proj(x)
        return logits
        